In [1]:
import os
import json
import base64
from PIL import Image
from io import BytesIO

def decode_base64(data):
    # Check if the data URI scheme is present and remove it
    if data.startswith('data:image/png;base64,'):
        data = data.replace('data:image/png;base64,', '')
    # Add padding if necessary to fix the "Incorrect padding" error
    padding = 4 - (len(data) % 4)
    if padding:
        data += '=' * padding
    return base64.b64decode(data)

def save_images_from_json(json_filepath, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load the JSON file
    with open(json_filepath, 'r') as file:
        data = json.load(file)

    # Process each category of images
    for category in data:
        category_path = os.path.join(output_dir, category)
        if not os.path.exists(category_path):
            os.makedirs(category_path)

        # Decode and save each image
        for i, base64_png in enumerate(data[category]):
            try:
                # Decode the base64 string to bytes, handling padding issues
                image_data = decode_base64(base64_png)
                # Try to open the image
                image = Image.open(BytesIO(image_data))
                # Save the image
                image.save(os.path.join(category_path, f'image_{i+1}.png'))
            except Exception as e:
                # Print diagnostic information
                print(f"Failed to decode or save image {i+1} in category '{category}': {e}")
                print(f"Base64 data starts with: {base64_png[:50]}")  # Print a longer portion if needed

# Example usage
json_filepath = 'data.json'  # Path to the JSON file
output_dir = 'outputs'  # Directory to save the images
save_images_from_json(json_filepath, output_dir)